In [3]:
"""Import all the relevant libraries."""

from mkd_blockchain import MKDBlockchain
from blockchain_utils import BlockchainUtils
import block
from pubsub import pub
from node import Node
from transaction import Transaction
from sensor_transaction import SensorTransaction
from transaction_pool import TransactionPool
from wallet import Wallet
from Crypto.PublicKey import RSA
import pprint
import random
import time
from cluster import Cluster
import kdtree
import copy

In [4]:
"""Initialize Nodes"""

print('START')
test_num = 1
cycles = 10
MAX_NODES = 9999
num_nodes = 50
num_clusters = 1
forging_interval = 2  # Time interval.  units of cycle
nodes = [] #list of all nodes
blockchain_dimensions = 4
genesis_node_id = int(num_nodes/2)

#itterate through and initialize each node
then = time.time()
for i in range(num_nodes):
    new_node = Node(test_num, i+1, f'{test_num}.c0')
    #new_node.blockchain = mkd_blockchain
    new_node.start_listener(f'{test_num}.c0')
    nodes.append(new_node)
now = time.time()-then
print(f'node init time @ {num_nodes} nodes: {now}')

START
node init time @ 50 nodes: 98.92878794670105


In [5]:
"""Initialize clusters"""
#  initialize each cluster
clusters = []
for i in range(num_clusters):
    new_cluster = Cluster(i, test_num)
    new_cluster.start_listener(f'{test_num}.c{i}')
    clusters.append(new_cluster)
    print(f'cluster {new_cluster.cluster_id} initialized')

FileNotFoundError: [Errno 2] No such file or directory: 'blockchain/keys/genesisPublicKey.pem'

In [ ]:
"""Create genesis block and publish to each node"""

genesis_forger = nodes[genesis_node_id-1].wallet.public_key_string()
mkd_blockchain = MKDBlockchain(blockchain_dimensions, genesis_node_id, genesis_forger)
publisher = nodes[genesis_node_id-1]
start_broadcast = time.time()
publisher.publish(mkd_blockchain)
completed_broadcast = time.time() - start_broadcast
print(f'time to complete broadcast of genesis block {mkd_blockchain.blocks.data} to {num_nodes} nodes: {completed_broadcast}')

In [ ]:
'''Create randomized list of node indexes'''

randomized_node_indexes = []
for i in range(num_nodes):
    randomized_node_indexes.append(i)
randomized_node_indexes: random.shuffle(randomized_node_indexes)
print(randomized_node_indexes)

In [ ]:
'''Randomly moves nodes into their initial clusters'''

nodes_available = len(nodes)
while nodes_available != 0:
    amount_to_move = random.randint(0,2)
    random_cluster = f'{test_num}.c{(random.randint(1,num_clusters))}'
    if nodes_available > 0 and nodes_available >= amount_to_move and nodes_available != 0:  # min 1 max 4
        for node in range(amount_to_move):
            nodes_available -= 1
            move_index = randomized_node_indexes.pop(0)
            nodes[move_index].move_node(nodes[move_index].cluster_id, random_cluster)

In [ ]:
'''Going to generate random transactions to publish at random intervals'''

# return true or false randomly
def gen_random():
    return bool(random.getrandbits(1))

# Traverses all nodes during each cycle and randomly creates transactions and moves nodes and forges a block at the chosen forge interval
for i in range(cycles):
    for node in nodes:
        if gen_random():  # Generate transactions randomly
            transaction = SensorTransaction(node.wallet.public_key_string(), random.randint(0,1000))
            node.publish(transaction)
        if gen_random():  # Moves the nodes randomly
            random_cluster = f'{test_num}.c{(random.randint(1,num_clusters))}'
            node.move_node(node.cluster_id, random_cluster)
        if i%forging_interval == 0:  # Forges at chosen forge interval in cycles
            for cluster in clusters:  # Each cluster forges a block at the chosen interval
                forger = cluster.next_forger()
                for cluster_node in cluster.member_nodes:
                    if cluster_node.wallet.public_key_string() == forger and cluster_node.transaction_pool.transactions is not []:
                        cluster_node.coords = [random.randint(1,100), random.randint(1,100)]
                        cluster_node.mkd_forge()

#  Need to move all nodes into the same cluster at the end and merge all chains
for node in nodes:
    random_cluster = f'{test_num}.c1'
    node.move_node(node.cluster_id, random_cluster)


print(f'Final Tree size: {clusters[0].member_nodes[0].blockchain.blocks.size} Left branch size: {clusters[0].member_nodes[0].blockchain.blocks.left_size} Right branch size: {clusters[0].member_nodes[0].blockchain.blocks.right_size}')
# print(f'Cluster mkd-tree blockchain: ')
# kdtree.visualize(clusters[0].member_nodes[0].blockchain.blocks)

In [ ]:
"""Forge random blocks at random intervals (need to code for specific forging triggers)"""

# number_of_forges = 4
#
# for i in range(number_of_forges):
#     random_node = random.randint(0, num_nodes-1)
#     random_move = random.randint(0, num_nodes-1)
#     # if 1 == random.randint(0,5):  # to cause random movement
#     #      random_move.node.move_node()
#     forge_node = nodes[random_node]
#     forge_node.mkd_forge()
#
# for node in nodes:
#     print(f'Cluster_id: {node.cluster_id} Node_id: {node.node_id} Tree size: {node.blockchain.blocks.size} Left size: {node.blockchain.blocks.left_size} Right size: {node.blockchain.blocks.right_size}')